In [27]:
import os
import fitz  # PyMuPDF for PDF extraction
import re
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from docx import Document
from pdf2docx import Converter  # Import for PDF to DOCX conversion
from comtypes.client import CreateObject  # For DOCX to PDF conversion

# Load API key from .env file
load_dotenv()
api_key_groq = os.getenv("GROQ_API_KEY")

# Define directories
DATA_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data"
OUTPUT_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output"

# Input file paths
prev_pdf_path = os.path.join(DATA_DIR, "prev_year_copy.pdf")
curr_pdf_path = os.path.join(DATA_DIR, "curr_year_report.pdf")

# Output file paths
prev_docx_path = os.path.join(OUTPUT_DIR, "prev_year_copy.docx")
blank_prev_docx_path = os.path.join(OUTPUT_DIR, "blank_prev_year.docx")
filled_docx_path = os.path.join(OUTPUT_DIR, "filled_prev_year.docx")
filled_pdf_path = os.path.join(OUTPUT_DIR, "filled_prev_year.pdf")

In [28]:

### 1️⃣ Convert Previous Year PDF to DOCX ###
def pdf_to_docx(input_pdf, output_docx):
    print(f"🔄 Converting {input_pdf} to {output_docx}...")
    cv = Converter(input_pdf)
    cv.convert(output_docx, start=0, end=None)  # Convert all pages
    cv.close()
    print("✅ PDF converted to DOCX with formatting!")

pdf_to_docx(prev_pdf_path, prev_docx_path)

[INFO] Start to convert C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data\prev_year_copy.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


🔄 Converting C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data\prev_year_copy.pdf to C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\prev_year_copy.docx...


[INFO] [3/4] Parsing pages...
[INFO] (1/1) Page 1
[INFO] [4/4] Creating pages...
[INFO] (1/1) Page 1
[INFO] Terminated in 0.38s.


✅ PDF converted to DOCX with formatting!


In [29]:
from docx import Document
import re

def replace_numbers_in_docx(input_docx, output_docx):
    doc = Document(input_docx)
    
    # Regex pattern to match numbers, percentages, and currency values
    number_pattern = re.compile(r"\b\d+[\.,]?\d*\b|\b\d{1,3}(?:,\d{3})*\b|\$\d+[\.,]?\d*|\d+%")

    for para in doc.paragraphs:
        for run in para.runs:  # Iterate through each formatted run
            if number_pattern.search(run.text):  # Check if the run contains numbers
                run.text = number_pattern.sub("__", run.text)  # Replace numbers only in that run

    doc.save(output_docx)
    print(f"✅ Replaced numbers in {input_docx}, saved as {output_docx} with formatting preserved.")

replace_numbers_in_docx(prev_docx_path, blank_prev_docx_path)


✅ Replaced numbers in C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\prev_year_copy.docx, saved as C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\blank_prev_year.docx with formatting preserved.


In [30]:


### 3️⃣ Extract Text from Current Year PDF ###
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text("text") for page in doc])

curr_year_text = extract_text_from_pdf(curr_pdf_path)

In [113]:
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.1, api_key=api_key_groq)

prompt_template = PromptTemplate(
    input_variables=["prev_report", "curr_report"],
    template="""
You are an expert financial analyst and document editor. Your task is to extract missing numerical values (`__`) from the previous year’s report using the current year’s report.

---

### **🚨 Critical Instructions**
1. **Strictly Extract Values in Order**  
   - ✅ Identify and extract **each number** in the current report in the **exact order** of placeholders (`__`).  
   - ✅ If a **number exists in the current report, copy it exactly** (no modifications).  
   - ✅ Preserve decimal values (e.g., `9.2` must remain `9.2`).  

2. **Handling Year References**  
   - ✅ Identify the **report year** from the current document heading.  
   - ✅ Replace references to **“last year” or “previous year”** with the correct year (e.g., if the report is for **2024**, then "last year" → **2023**).  
   - ✅ Ensure the **year appears in the correct place** (especially at the start).

3. **Ensure Every Placeholder is Filled**  
   - ✅ Each `__` in the previous report **must be replaced in the exact order**.  
   - ✅ No missing values.  
   - ✅ Maintain original formatting.  

---

### **🔹 Inputs**
#### **Previous Year’s Report (with missing values)**
{prev_report}

#### **Current Year’s Report (reference data)**
{curr_report}

---

### **🔹 Expected Output Format**
Return **only** a valid Python list of numbers in the **exact order**:  


    """
)

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

blank_prev_year_text = extract_text_from_docx(blank_prev_docx_path)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)
response = llm_chain.invoke({"prev_report": blank_prev_year_text, "curr_report": curr_year_text})



[INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [114]:
print(response['text'])

[2023, 9.8, 24.7, 9.8, 24.7, 15.1, 1.6, 32, 2, 33, 3, 1.5, 33, 10]


In [33]:
from docx import Document

def replace_placeholders_in_docx(input_docx, output_docx, replacements):
    """
    Replaces placeholders (____) in a DOCX file with provided values while keeping formatting intact.
    """
    doc = Document(input_docx)
    placeholder_count = 0
    replacements = replacements.split("\n")  # Convert response to list of values

    for para in doc.paragraphs:
        full_text = "".join(run.text for run in para.runs)  # Get full paragraph text
        
        # Only proceed if the paragraph contains placeholders
        if "__" in full_text and placeholder_count < len(replacements):
            updated_text = full_text
            while "__" in updated_text and placeholder_count < len(replacements):
                updated_text = updated_text.replace("__", replacements[placeholder_count], 1)
                placeholder_count += 1

            # Update the runs while preserving formatting
            current_index = 0
            for run in para.runs:
                run_length = len(run.text)
                run.text = updated_text[current_index:current_index + run_length]  # Assign formatted portions back
                current_index += run_length

    doc.save(output_docx)
    print(f"✅ Updated placeholders in {input_docx}, saved as {output_docx}")

# Extract the filled values from LLM response
filled_values = response["text"]

# Run the function
replace_placeholders_in_docx(blank_prev_docx_path, filled_docx_path, filled_values)


✅ Updated placeholders in C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\blank_prev_year.docx, saved as C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\filled_prev_year.docx
